In [1]:
%pip install lxml
%pip install MyCapytain
%pip install pandas
%pip install 'spacy[apple]'
%pip install grecy
%run -m spacy download en_core_web_sm


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation succes


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# Import libraries
from MyCapytain.resources.texts.local.capitains.cts import CapitainsCtsText
from lxml import etree
from MyCapytain.common.constants import Mimetypes
import pandas as pd
import spacy
from collections import Counter

In [3]:
def get_book_of_pausanias(df: pd.DataFrame, book_n: int):
    return df[df['urn'].str.startswith(f"urn:cts:greekLit:tlg0525.tlg001.perseus-eng2:{book_n}")]

In [23]:
# Load Pausanias text 
# book = ['tlg0525.tlg001.perseus-eng2.xml']
# book = get_book_of_pausanias(pausanias_df, 2)

with open("../tei/tlg0525.tlg001.perseus-eng2.xml") as f:
    text = CapitainsCtsText(urn="urn:cts:greekLit:tlg0525.tlg001.perseus-eng2", resource=f)

urns = []
raw_xmls = []
unannotated_strings = []

for ref in text.getReffs(level=len(text.citation)):
    urn = f"{text.urn}:{ref}"
    node = text.getTextualNode(ref)
    raw_xml = node.export(Mimetypes.XML.TEI)
    tree = node.export(Mimetypes.PYTHON.ETREE)
    s = etree.tostring(tree, encoding="unicode", method="text")

    urns.append(urn)
    raw_xmls.append(raw_xml)
    unannotated_strings.append(s)


In [5]:
# Create a DataFrame to store all the data from the three books
d = {
    "urn": pd.Series(urns, dtype="string"),
    "raw_xml": raw_xmls,
    "unannotated_strings": pd.Series(unannotated_strings, dtype="string")
}
pausanias_df = pd.DataFrame(d)

# Tokenize and analyze the text
pausanias_df['whitespaced_tokens'] = pausanias_df['unannotated_strings'].str.split()


In [6]:
nlp = spacy.load("en_core_web_sm")

pausanias_df['tokens'] = pausanias_df['unannotated_strings'].apply(nlp.tokenizer)

raw_texts = [t for t in pausanias_df['unannotated_strings']]
annotated_texts = nlp.pipe(raw_texts, batch_size=100)

pausanias_df['nlp_docs'] = list(annotated_texts)


/opt/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [34]:
# BOOK 2
book_2 = get_book_of_pausanias(pausanias_df, 2)

# TOKENS

tokens = [t.text for t in book_2['tokens'].explode() if t.is_alpha]
# tokens = [t.text for t in get_book_of_pausanias(pausanias_df, 2)['tokens'].explode() if t.is_alpha]

token_counts = Counter(tokens)

token = token_counts.most_common(100)

# TYPES

types = [t.text for t in book_2['tokens'].explode() if not t.is_stop and t.is_alpha]

type_counts = Counter(types)

type = type_counts.most_common(100)

# LEMMATA

lemmata = [t.lemma_ for t in book_2['nlp_docs'].explode() if not t.is_stop and t.is_alpha]

lemmata_counts = Counter(lemmata)

lemmata = lemmata_counts.most_common(100)


token_series = pd.Series([t for t, _ in token_counts.most_common()], name="Token")
type_series = pd.Series([t for t, _ in type_counts.most_common()], name="Type")
lemma_series = pd.Series([t for t, _ in lemmata_counts.most_common()], name="Lemma")

combined_df = pd.concat([token_series, type_series, lemma_series], axis=1)

# #Dont truncate dataframe
# pd.set_option('display.max_rows', None)  # Show all rows

# # Print df
# combined_df

# # Print the content of Book 2
# def get_book_of_pausanias(df: pd.DataFrame, book_n: int):
#     return df[df['urn'].str.startswith(f"urn:cts:greekLit:tlg0525.tlg001.perseus-eng2:{book_n}")]
# book_2 = get_book_of_pausanias(pausanias_df, 2)
# book_2_text = " ".join(book_2['unannotated_strings'])

# print(book_2_text)



In [20]:
# BOOK 3
book_3 = get_book_of_pausanias(pausanias_df, 3)

# TOKENS

tokens = [t.text for t in book_3['tokens'].explode() if t.is_alpha]
# tokens = [t.text for t in get_book_of_pausanias(pausanias_df, 2)['tokens'].explode() if t.is_alpha]

token_counts = Counter(tokens)

token = token_counts.most_common(100)

# TYPES

types = [t.text for t in book_3['tokens'].explode() if not t.is_stop and t.is_alpha]

type_counts = Counter(types)

type = type_counts.most_common(100)

# LEMMATA

lemmata = [t.lemma_ for t in book_3['nlp_docs'].explode() if not t.is_stop and t.is_alpha]

lemmata_counts = Counter(lemmata)

lemmata = lemmata_counts.most_common(100)

token_series = pd.Series([t for t, _ in token_counts.most_common()], name="Token")
type_series = pd.Series([t for t, _ in type_counts.most_common()], name="Type")
lemma_series = pd.Series([t for t, _ in lemmata_counts.most_common()], name="Lemma")

combined_df = pd.concat([token_series, type_series, lemma_series], axis=1)

combined_df

,Token,Type,Lemma
0,the,son,son
1,of,Lacedaemonians,Lacedaemonians
2,and,sanctuary,sanctuary
3,to,called,call
4,a,image,image
...,...,...,...
3617,Calathium,NaN,NaN
3618,Claea,NaN,NaN
3619,narrow,NaN,NaN
3620,Alagonia,NaN,NaN


In [21]:
# BOOK 4
book_4 = get_book_of_pausanias(pausanias_df, 4)

# TOKENS

tokens = [t.text for t in book_4['tokens'].explode() if t.is_alpha]
# tokens = [t.text for t in get_book_of_pausanias(pausanias_df, 2)['tokens'].explode() if t.is_alpha]

token_counts = Counter(tokens)

token = token_counts.most_common(100)

# TYPES

types = [t.text for t in book_4['tokens'].explode() if not t.is_stop and t.is_alpha]

type_counts = Counter(types)

type = type_counts.most_common(100)

# LEMMATA

lemmata = [t.lemma_ for t in book_4['nlp_docs'].explode() if not t.is_stop and t.is_alpha]

lemmata_counts = Counter(lemmata)

lemmata = lemmata_counts.most_common(100)

token_series = pd.Series([t for t, _ in token_counts.most_common()], name="Token")
type_series = pd.Series([t for t, _ in type_counts.most_common()], name="Type")
lemma_series = pd.Series([t for t, _ in lemmata_counts.most_common()], name="Lemma")

combined_df = pd.concat([token_series, type_series, lemma_series], axis=1)

combined_df

,Token,Type,Lemma
0,the,Messenians,Messenians
1,of,Lacedaemonians,Lacedaemonians
2,and,son,son
3,to,Aristomenes,man
4,in,men,come
...,...,...,...
3902,Dionysias,NaN,NaN
3903,Cyparissia,NaN,NaN
3904,depression,NaN,NaN
3905,Aulon,NaN,NaN


2. The tokens, such as 'the', 'of', 'and', 'to', and 'in', reflect high-frequency words typically used for sentence structure rather than conveying specific meaning. This is expected in most texts, as function words like articles, prepositions, and conjunctions are necessary for syntactic coherence but don't provide much insight into the content itself.

The types, such as 'son', 'sanctuary', 'city', and 'Zeus', highlight key themes in Pausanias' text—namely, discussions of familial relations, religious sites, and deities. These types give us a clearer understanding of the subject matter of the text, emphasizing locations and mythological figures that were important to the ancient Greeks.

When we look at the lemmata, the patterns become even more informative. Words like 'call', 'come', 'say', and 'give' reflect common actions, while proper nouns like 'Zeus', 'Apollo', and 'Athena' further emphasize the religious and mythological context of Pausanias' work. The lemmatization process removes inflections, allowing us to see the underlying forms of words, which helps highlight the core concepts and actions described.

While the tokens give a general picture of the text's language structure, the types and lemmata offer deeper insights into the themes and central figures in Pausanias' narrative. This breakdown provides a clearer understanding of how the linguistic and thematic elements are intertwined in the text.

3. While quantitative analysis identifies the frequency of specific words and phrases, it often lacks the ability to convey the complexities of human emotions and experiences depicted in the text. For example, while terms like "sacrifice" may appear frequently, a qualitative reading allows for an exploration of the emotional weight behind these rituals—such as the tension between devotion and loss, or the anxiety surrounding the acceptance of divine favor.

Qualitative reading allows for an examination of Pausanias' narrative style, tone, and perspective. Understanding his biases, motivations, and the emotional undertones of his writing provides insight into how he shapes the reader's perception of ancient Greece. This dimension of analysis is often absent from quantitative methods.
    As a Greek author, Pausanias writes from a distinctly Hellenic perspective. His admiration for Greek culture and values is evident throughout his work. This bias may lead him to glorify Greek achievements—especially in religion, athletics, and art—while potentially downplaying or criticizing non-Greek customs. For example, he often emphasizes the superiority of Greek gods and rituals over those of other cultures, reflecting a sense of cultural pride.
    Throughout his work, Pausanias exhibits a reflective tone as he contemplates the meanings behind the monuments and rituals he describes. He often pauses to provide insights into their historical significance or to muse on their cultural implications. This reflective quality invites readers to engage with the text on a deeper level, encouraging them to consider the complexities of Greek identity.
    Pausanias' writing is imbued with emotional undertones, especially when he reflects on loss, decay, and the passage of time. His descriptions of ruined temples or forgotten traditions often evoke a sense of melancholy, prompting readers to contemplate the fragility of cultural heritage. This emotional resonance deepens the reader's connection to the text, inviting them to share in Pausanias' appreciation and sorrow.
All of these may not be initially evident from a quantitative analysis alone.

In Book 2 of Description of Greece, Pausanias provides a detailed account of the ancient city of Corinth, its temples, and the associated rituals. While a quantitative analysis may highlight the frequency of terms such as "temple," "sacrifice," or "Athena," it cannot capture the emotional weight of Pausanias' reflections on the significance of these structures and practices. For instance, his description of the temple of Apollo not only outlines its architectural grandeur but also conveys a palpable reverence that stems from its historical and cultural importance. Pausanias evokes a sense of nostalgia for the divine connection that these sites represent, which contrasts sharply with the mere counting of words.

Additionally, Pausanias’ examination of the sacred sites in Corinth serves to illustrate his biases and motivations more clearly. His admiration for Greek culture is particularly pronounced in his portrayal of the various festivals and athletic competitions held in honor of the gods, where he not only details the events but also emphasizes their role in fostering communal identity and continuity. Through his reflective tone, Pausanias invites readers to appreciate the deep-rooted traditions that bind the people of Corinth to their heritage, highlighting the emotional stakes involved in their religious practices. This nuanced understanding of the text allows for a richer interpretation of the significance of these rituals, transcending what a quantitative approach could reveal.
